## L&T EduTech Hackathon: Problem Statement 1
Deep Learning framework for Crack Detection

In [ ]:
import os
import numpy as np

import tensorflow as tf
from tensorflow.keras import applications
from tensorflow.keras.utils import image_dataset_from_directory

import matplotlib.pyplot as plt

In [ ]:
# Config Variables
DIR = "./Surface Crack" # Replace with Path to Data Folder

## Load Data

In [ ]:
def load_images(directory, subset, batch_size=32, image_size=(224, 224), shuffle=False):
    directory = os.path.join(directory, subset)
    return image_dataset_from_directory(
        directory,
        labels="inferred",
        label_mode="int",
        class_names=['Negative','Positive'],
        color_mode="rgb",
        batch_size=batch_size,
        image_size=image_size,
        shuffle=shuffle,
    ).cache().prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# No shuffling to preserve order
train_ds = load_images(DIR, 'train')
print("Number of training samples: %d" % tf.data.experimental.cardinality(train_ds))
val_ds = load_images(DIR, 'valid')
print("Number of validation samples: %d" % tf.data.experimental.cardinality(val_ds))
test_ds = load_images(DIR, 'test')
print("Number of testing samples: %d" % tf.data.experimental.cardinality(test_ds))

Found 600 files belonging to 2 classes.
Number of training samples: 19
Found 200 files belonging to 2 classes.
Number of validation samples: 7
Found 200 files belonging to 2 classes.
Number of testing samples: 7


## Deep Learning Model

In [ ]:
class KerasModel():
    def __init__(self, name, instance, preprocess_fn, subtype_name=None, subtype_module=None):
        self.name = name
        self.instance = instance
        self.preprocess_fn = preprocess_fn
        self.subtype_name = subtype_name
        self.subtype_module = subtype_module

    def __call__(self, **kwargs):
        return instance(**kwargs)

In [ ]:
# Handles all models except MobileNetV3Small & MobileNetV3Large
model_subtypes = [model_subtype for model_subtype in dir(applications) if model_subtype[0].islower()]
models = []

for model_subtype in model_subtypes:
    model_subtype_mod = getattr(applications, model_subtype)
    model_names = [model_name for model_name in dir(model_subtype_mod) if model_name[0].isupper()]
    preprocess_fn = getattr(model_subtype_mod, "preprocess_input")
    for model_name in model_names:
        instance = getattr(model_subtype_mod, model_name)
        models.append(KerasModel(model_name, instance, preprocess_fn))

print(f"{len(models)} models from the {len(model_subtypes)} model types.")

65 models from the 18 model types.


In [ ]:
!mkdir embeddings

In [ ]:
INPUT_SHAPE = (224, 224, 3)

for model_obj in models:
    tf.keras.backend.clear_session()
    print(model_obj.name)
    base_model = model_obj(
        weights="imagenet",
        input_shape=INPUT_SHAPE,
        include_top=False,
        pooling='avg'
    )

    # Freeze the base_model
    base_model.trainable = False

    # Create new model on top
    inputs = tf.keras.Input(shape=INPUT_SHAPE)

    # Pre-trained Xception weights requires that input be preprocessed
    x = model_obj.preprocess_fn(inputs)

    # The base model contains batchnorm layers. We want to keep them in inference mode
    # when we unfreeze the base model for fine-tuning, so we make sure that the
    # base_model is running in inference mode here.
    embeddings = base_model(x, training=False)
    model = tf.keras.Model(inputs, embeddings)

    train_pred = model.predict(train_ds, verbose=1)
    val_pred = model.predict(val_ds, verbose=1)
    test_pred = model.predict(test_ds, verbose=1)
    np.savez_compressed(f"embeddings/{model_obj.name}.npz",train=train_pred,val=val_pred,test=test_pred)

DenseNet121
7/7 [==============================] - 1s 109ms/step
DenseNet169
7/7 [==============================] - 1s 110ms/step
DenseNet201
7/7 [==============================] - 1s 111ms/step
EfficientNetB0
7/7 [==============================] - 1s 108ms/step
EfficientNetB1
7/7 [==============================] - 1s 108ms/step
EfficientNetB2
7/7 [==============================] - 1s 108ms/step
EfficientNetB3
7/7 [==============================] - 1s 110ms/step
EfficientNetB4
7/7 [==============================] - 1s 111ms/step
EfficientNetB5
7/7 [==============================] - 1s 111ms/step
EfficientNetB6
7/7 [==============================] - 1s 110ms/step
EfficientNetB7
7/7 [==============================] - 1s 111ms/step
EfficientNetV2B0
7/7 [==============================] - 1s 112ms/step
EfficientNetV2B1
7/7 [==============================] - 1s 112ms/step
EfficientNetV2B2
7/7 [==============================] - 1s 114ms/step
EfficientNetV2B3
7/7 [=============================

In [ ]:
# Copy to Drive
!cp -r "/content/embeddings" "/content/drive/MyDrive/Kaggle Dataset/Surface Crack"